In [1]:
import pandas as pd

In [7]:
def replenish_ip(ip):
    '''
    补全ip中的 :: 为0000
    补全ip中前导0的省略
    '''
    org_length = 8
    splited_ = ip.split(':')
    if '' in splited_:
        length = len(splited_) - 1
    else:
        length = len(splited_)
    d = org_length - length
    if ip.endswith('::'):
        ip = ip.replace('::', ':0000'*d)
    else:
        ip = ip.replace('::', ':0000'*d + ':')
    temp = ip.split(':')[4:]
    # print(temp)
    for index, i in enumerate(temp):
        l = len(i)
        if l < 4:
            d = 4 - l
            temp[index] = '0'*d + i

    return ''.join(temp)

In [8]:
replenish_ip('2400:cb00:2048:1::6810:a870')

'000000006810a870'

In [2]:
EUI_path = r'D:\data\EUI-64.csv'
DHCP_path = r'D:\data\DHCP.xlsx'
SLAAC_path = r'D:\data\SLAAC.xlsx'

In [42]:
from math import log2
import numpy as np

def cal_entropy(sq):
    '''
    哎，这种东西还是自己写好b（￣▽￣）d　
    '''
    times_dict = dict()
    _set = set(sq)
    for i in _set:
        times_dict[i] = 0
    for key in times_dict.keys():
        for i in sq:
            if key == i:
                times_dict[i] += 1
    _sum = len(sq)
    ent = 0
    for i in _set:
        p = times_dict[i] / _sum
        # print(p)
        ent += p*log2(p)

    return - ent


def cal_aver_ent(path):
    if path.endswith('.csv'):
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)
    content = list(df['ip'])
    
    ent_arr = np.array([])
    for ip in content:
        interface = list(replenish_ip(ip))
        ent_1 = cal_entropy(interface[:4])
        ent_2 = cal_entropy(interface[4:8])
        ent_3 = cal_entropy(interface[8:12])
        ent_4 = cal_entropy(interface[12:16])
        ent = (ent_1 + ent_2 + ent_3 + ent_4)
        ent_arr = np.append(ent_arr, ent)
    
    return (df, ent_arr)
        
print(cal_aver_ent(DHCP_path)[1].mean())
# print(cal_aver_ent(EUI_path)[1].mean())
print(cal_aver_ent(SLAAC_path)[1].mean())

3.2611692056598915
7.225773878141035


In [10]:
cal_entropy(list('023048fffe560f93'))

3.077819531114783

In [43]:
df, DHCP_ent_arr = cal_aver_ent(DHCP_path)
df['entropy'] = DHCP_ent_arr
df

,ip,entropy
0,2001:1210:100:1::17,1.500000
1,2001:1210:100:1::218,2.000000
2,2001:1210:105:34:0:403:a8:1,3.811278
3,2001:1210:7000:85a::18,1.500000
4,2001:1210:7000:85b::2,0.811278
5,2001:1210::1:1,1.622556
6,2001:1210::1:5,1.622556
7,2001:1210::1:6,1.622556
8,2001:1210::1:d,1.622556
9,2001:1218:1000:230::241,2.000000


In [33]:
df.describe()

,entropy
count,1960.000000
mean,1.610393
std,0.556579
min,0.337290
25%,1.186278
50%,1.621641
75%,2.091917
max,2.905639


上面那个是之前的

In [44]:
df.describe()

,entropy
count,1960.000000
mean,3.261169
std,1.313254
min,-0.000000
25%,2.311278
50%,3.122556
75%,4.000000
max,7.500000


In [45]:
_df, SLAAC_ent_arr = cal_aver_ent(SLAAC_path)
_df['entropy'] = SLAAC_ent_arr
_df.min()

ip         2001:1210:7000:85a:61bb:f3ee:680b:e491
entropy                                         3
dtype: object

In [46]:
_df.describe()

,entropy
count,2170.000000
mean,7.225774
std,0.667444
min,3.000000
25%,7.000000
50%,7.500000
75%,7.500000
max,8.000000


In [73]:
df[df['entropy']<6.3].count()

ip         1883
entropy    1883
dtype: int64

In [74]:
df[df['entropy']>6.3].count()

ip         77
entropy    77
dtype: int64

In [75]:
_df[_df['entropy']>6.3].count()

ip         2063
entropy    2063
dtype: int64

In [76]:
_df[_df['entropy']<6.3].count()

ip         107
entropy    107
dtype: int64

In [77]:
df[df['entropy']>6.3]

,ip,entropy
432,2001:1a68:b:1:2:149:198:57,6.311278
435,2001:1a68:b:1:4:149:198:42,6.311278
436,2001:1a68:b:1:4:149:198:43,6.311278
437,2001:1a68:b:1:4:149:198:46,6.311278
438,2001:1a68:b:1:4:149:198:47,6.311278
545,2001:288:1033:91:192:192:91:191,7.000000
546,2001:288:1033:91:192:192:91:193,7.500000
547,2001:288:1033:91:192:192:91:194,7.500000
548,2001:288:1033:91:192:192:91:195,7.500000
549,2001:288:1033:91:192:192:91:196,7.500000
